<a href="https://colab.research.google.com/github/ag826/AIPI590_XAI_F25/blob/main/Assignment_Explainable_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment - Explainable deep Learning
**Adil Keku Gazder**

**ag825@duke.edu**

**For AIPI 590 - XAI, Fall 2025**

**Duke University**




We work with a pretrained deep learning models to investigate model explainability in computer vision. Our objective is to apply GradCAM and at least two of its variants to a meaningful image classification problem of your choice and analyze how and why the model makes its decisions.

For this assignment, we choose to try and classify satellite images into different categories. We use the EuroSat dataset and the ResNet-50 model. We also do use GradCAM, GradCAM++ and ScoreCAM for the explainability aspect.


In [1]:
# Install the library for GradCAM
!pip install grad-cam

# Import necessary packages
import torch
from torchvision import models, transforms
from torchvision.datasets import EuroSAT
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random

# Import the CAM techniques
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.5.5-py3-none-any.whl size=44284 sha256=02ce3fea1b059eb5134ad8d91984137da7774a9ff9b81ee948bee16d47171aff
  Stored in directory: /root/.cache/pip/wheels/fb/3b/09/2afc520f3d69bc26ae6bd87416759c820a3f7d05c1a077bbf6
Successfully built grad-cam


In [2]:
# Load a pretrained model (ResNet-50)
model = models.resnet50(pretrained=True)
model.eval() # Set the model to evaluation mode

# Define the target layer for ResNet-50's CAM
target_layers = [model.layer4[-1]]

# Define the image transformations required by ResNet-50
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Download and load the EuroSAT dataset
eurosat_dataset = EuroSAT(root="data", download=True, transform=transform)
class_names = eurosat_dataset.classes
print("EuroSAT Classes:", class_names)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:01<00:00, 94.7MB/s]
100%|██████████| 94.3M/94.3M [00:00<00:00, 162MB/s]


EuroSAT Classes: ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']


In [ ]:

# --- Helper function: un-normalize an image tensor for visualization ---
def un_normalize(tensor):
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    tensor = tensor.clone().detach().cpu().numpy().squeeze().transpose((1, 2, 0))
    tensor = std * tensor + mean
    tensor = np.clip(tensor, 0, 1)
    return tensor

# --- Randomly select 5 different classes ---
unique_labels = list(set([label for _, label in eurosat_dataset]))
selected_labels = random.sample(unique_labels, 5)

# --- Initialize CAM methods once ---
cam_gradcam = GradCAM(model=model, target_layers=target_layers)
cam_gradcam_plusplus = GradCAMPlusPlus(model=model, target_layers=target_layers)
cam_scorecam = ScoreCAM(model=model, target_layers=target_layers)

# --- Set up plotting ---
fig, axs = plt.subplots(5, 4, figsize=(16, 20))
plt.subplots_adjust(hspace=0.4)
fig.suptitle("GradCAM Visualizations for 5 Different EuroSAT Classes", fontsize=18)

# --- Loop over selected classes ---
for row, label in enumerate(selected_labels):
    # Find an image of this class
    class_name = class_names[label]
    class_indices = [i for i, (_, lbl) in enumerate(eurosat_dataset) if lbl == label]
    img, lbl = eurosat_dataset[class_indices[0]]  # Take the first image for simplicity

    # Prepare the input tensor
    input_tensor = img.unsqueeze(0)
    rgb_img = un_normalize(img)

    # Model prediction and target
    prediction_idx = model(input_tensor).argmax()
    targets = [ClassifierOutputTarget(prediction_idx)]

    # Generate grayscale CAMs
    grayscale_gradcam = cam_gradcam(input_tensor=input_tensor, targets=targets)[0, :]
    grayscale_gradcam_plusplus = cam_gradcam_plusplus(input_tensor=input_tensor, targets=targets)[0, :]
    grayscale_scorecam = cam_scorecam(input_tensor=input_tensor, targets=targets)[0, :]

    # Overlay heatmaps
    vis_gradcam = show_cam_on_image(rgb_img, grayscale_gradcam, use_rgb=True)
    vis_gradcam_plusplus = show_cam_on_image(rgb_img, grayscale_gradcam_plusplus, use_rgb=True)
    vis_scorecam = show_cam_on_image(rgb_img, grayscale_scorecam, use_rgb=True)

    # --- Plot results for this image ---
    axs[row, 0].imshow(rgb_img)
    axs[row, 0].set_title(f"Original: {class_name}")
    axs[row, 0].axis('off')

    axs[row, 1].imshow(vis_gradcam)
    axs[row, 1].set_title("GradCAM")
    axs[row, 1].axis('off')

    axs[row, 2].imshow(vis_gradcam_plusplus)
    axs[row, 2].set_title("Grad-CAM++")
    axs[row, 2].axis('off')

    axs[row, 3].imshow(vis_scorecam)
    axs[row, 3].set_title("ScoreCAM")
    axs[row, 3].axis('off')

plt.show()

 19%|█▉        | 24/128 [01:33<06:39,  3.84s/it]